# Using formulas to specify models

All of the models can be specified using formulas. The formulas used here utilize [patsy](http://patsy.readthedocs.io/en/latest/) are similar to those in [statsmodels](http://www.statsmodels.org).  The basis formula syntax for a single variable regression would be

```
y ~ 1 + x
```

The formulas used with ``BetweenOLS``, ``PooledOLS`` and ``RandomEffects`` are completely standard and are identical to [statsmodels](http://www.statsmodels.org).  ``FirstDifferenceOLS`` is nearly identical with the caveat that the model *cannot* include an intercept. 

``PanelOLS``, which implements effects (entity, time or other) has a small extension to the formula which allows entity effects or time effects (or both) to be specified as part of the formula. While it is not possible to specify other effects using the formula interface, these can be included as an optional parameter when using a formula.

## Loading and preparing data 
When using formulas, a MultiIndex pandas dataframe where the index is entity-time is **required**. Here the Grunfeld data, from "The Determinants of Corporate Investment", provided by [statsmodels](http://www.statsmodels.org/stable/datasets/generated/grunfeld.html), is used to illustrate the use of formulas.  This dataset contains data on firm investment, market value and the stock of plant capital. 

``set_index`` is used to set the index using variables from the dataset.

In [ ]:
from statsmodels.datasets import grunfeld
data = grunfeld.load_pandas().data
data = data.set_index(['firm','year'])
print(data.head())

## PanelOLS with Entity Effects

Entity effects are specified using the special command `EntityEffects`.  By default a constant is not included, and so if a constant is desired, `1+` should be included in the formula.  When including effects, the model and fit are identical whether a constant is included or not.

In [ ]:
from linearmodels import PanelOLS
mod = PanelOLS.from_formula('invest ~ value + capital + EntityEffects', data=data)
print(mod.fit())

## PanelOLS with Entity Effects and a constant

The constant can be explicitly included using the `1 + ` notation.  When a constant is included in the model, and additional constraint is imposed that the num of the effects is 0.  This allows the constant to be identified using teh grand mean of the dependent and the regressors.

In [ ]:
mod = PanelOLS.from_formula('invest ~ 1 + value + capital + EntityEffects', data=data)
print(mod.fit())

## Panel with Entity and Time Effects

Time effects can be similarly included using `TimeEffect`.  In many models, time effects can be consistently estimated and so they could be equivalently included in the set of regressors using a categorical variable.

In [ ]:
mod = PanelOLS.from_formula('invest ~ 1 + value + capital + EntityEffects + TimeEffects', data=data)
print(mod.fit())

## Between OLS

The other panel models are straight-forward and are included for completeness.

In [ ]:
from linearmodels import BetweenOLS, FirstDifferenceOLS, PooledOLS
mod = BetweenOLS.from_formula('invest ~ 1 + value + capital', data=data)
print(mod.fit())

## First Difference OLS

The first difference model must never include a constant since this is not identified after differencing.

In [ ]:
mod = FirstDifferenceOLS.from_formula('invest ~ value + capital', data=data)
print(mod.fit())

## Pooled OLS

The pooled OLS estimator is a special case of `PanelOLS` when there are no effects.  It is effectively identical to `OLS` in `statsmodels` (or `WLS`) but is included for completeness.

In [ ]:
mod = PooledOLS.from_formula('invest ~ 1 + value + capital', data=data)
print(mod.fit())